# Installing Marlowe CLI and Associated Tools

This lecture describes how to install `marlowe-cli` and the other tools needed to run Marlowe contracts.

👉 Installation assistance is available on the Marlowe Discord channel <<https://discord.com/channels/826816523368005654/936295815926927390/936316494042779698>>.

## Prerequisites

In addition to `marlowe-cli` and other Cardano tools, the following common command-line tools are used in these lectures. Install these if they aren't already installed.

*   `bash`
*   `git`
*   `curl`
*   `sed`
*   `tr`
*   `basenc`
*   `jq`

## Cloning the Git Repository for Marlowe

The Marlowe repository for Cardano is located at <<https://github.com/input-output-hk/marlowe-cardano.git>>.
*   This repository contains the Marlowe source code, documentation, examples, and test cases.
*   The repository is needed if you want to use Marlowe from a Nix shell.
*   It is convenient, but not strictly required, if you install Marlowe tools from the Marlowe Pioneers Daedalus installation.

The following `git` command will copy the repository onto your machine.

In [1]:
git clone git@github.com:input-output-hk/marlowe-cardano.git -b mpp-cli-lectures

Cloning into 'marlowe-cardano'...
remote: Enumerating objects: 127844, done.
remote: Counting objects: 100% (617/617), done.
remote: Compressing objects: 100% (327/327), done.
remote: Total 127844 (delta 294), reused 583 (delta 276), pack-reused 127227
Receiving objects: 100% (127844/127844), 119.44 MiB | 7.89 MiB/s, done.
Resolving deltas: 100% (81794/81794), done.
Note: switching to '0951b134e7744f1a4fafc8efbd0d2d1ddbba8e46'.


## Installating Daedalus

Daedalus is the Cardano full-node wallet. The version for Marlowe Pioneers runs a Cardano node on the Marlowe Pioneers test network.

1.  Download the version of Daedalus for your operating system from <<https://buildkite.com/input-output-hk/daedalus/builds?branch=feature%2Fddw-1039-marlowe-pioneers-installer>>.
2.  Run the Daedalus installer.
3.  Launch Daedalus from the menu of programs installed in your operating system.

🤔 ***Do we want to include screenshots of Daedalus in this slide and in subsequent slides?***

## Creating a Cardano Wallet using Daedalus

1.  Click "Add wallet" on the main screen.
2.  Click "Create".
3.  Enter a nickname for the wallet and a password.
4.  Daedalus will display a seed phrase and instructions for securing it. Keep a record of this seed phrase so that you can use the wallet at the command line or in Marlowe Run.

If you previously created a wallet using Marlowe Run, instead of creating a new wallet you can use the "Restore" function in place of "Create".

🚫 Never reuse the seed phrase from any `mainnet` wallet on a test network like the Marlowe Pioneers testnet.

🤔 ***Is this a strong enough warning, or should we say more?***

## Installing Marlowe CLI and Cardano Tools

NixOS and the Nix package manager are available at <<https://nixos.org/>>.
 
👉 See <<https://github.com/input-output-hk/marlowe-cardano/blob/main/README.adoc#how-to-set-up-the-iohk-binary-caches>> on how to set up binary caches for the Nix build. This will greatly speed the build process.

Once the binary caches have been set up, simply enter a nix shell.

In [2]:
nix-shell marlowe-cardano/shell.nix

The `marlowe-cli` and the other required executable programs are all available within that shell.

In [3]:
marlowe-cli --version

marlowe-cli 0.0.3.1 @ 


In [4]:
cardano-cli --version

cardano-cli 1.34.0 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [5]:
cardano-wallet version

v2022-01-18 (git revision: 9d5ab5ea5e51b4fab7e77cd1af6bc00822fe9d77)


## Creating a Payment and Signing Keys

In order to sign transactions in `marlowe-cli`, we need the payment and signing keys for the first address in the wallet. In this example, let's assume that the seed phrase for the wallet is stored in a file name `my-wallet.seed`. (The seed phrase could have been generated by Daedalus, by Marlowe, or by the command `cardano-wallet recovery-phrase generate`.) The following sequence of commands, which much be executed within the Nix shell for Marlowe CLI, will create payment and signing keys.

In [6]:
# View the seed phrase.
cat my-wallet.seed

broccoli tool napkin scale lab liquid staff turn equal city sail company govern hold rent act nurse orbit torch normal update master valley twenty


In [7]:
# Store the payment signing key in the file `my-walley.skey`.
cat my-wallet.seed | cardano-wallet key from-recovery-phrase Shelley | cardano-wallet key child 1852H/1815H/0H/0/0 > my-wallet.prv
cardano-cli key convert-cardano-address-key --shelley-payment-key --signing-key-file my-wallet.prv --out-file my-wallet.skey
rm my-wallet.prv

# Store the payment verification key in the file `my-wallet.vkey`.
cardano-cli key verification-key --signing-key-file my-wallet.skey --verification-key-file my-wallet.vkey

# Compute the address.
cardano-cli address build --testnet-magic 1567 --payment-verification-key-file my-wallet.vkey > my-wallet.address

Here are the keys:

In [8]:
ls -l my-wallet.skey my-wallet.vkey

-rw------- 1 pioneer marlowe 367 Apr 29 07:39 my-wallet.skey
-rw------- 1 pioneer marlowe 244 Apr 29 07:39 my-wallet.vkey


Here is the address:

In [9]:
cat my-wallet.address

addr_test1vp8fta3c0g85dd9pu4fp4tsjq955w2zz76y2ywumsmky86s5vzhzg

## Funding the Address of the Daedalus Wallet

You should have received an API key for the Marlowe Pioneers faucet. Call this `FAUCET_KEY`.

Run the following command to request test ADA.

In [10]:
curl -k -X POST https://faucet.pioneers.testnet.marlowe-finance.io/send-money/$(cat my-wallet.address)?apiKey=$FAUCET_KEY

{"success":true,"amount":1000000000000,"unit":"lovelace","fee":167965,"minLovelace":999978,"txid":"d8c062e3d350723938cae4f1708c4c5521ad349cd345c923f52f2ccba87fe311"}

In a minute or so, Daedalus should list the transaction that sent the funds and update the wallet's balance.

## Finding the Cardano Node Socket

Start Daedalus, which will run a Cardano node on the Marlowe Pioneers test network. Some `marlowe-cli` commands require a connection to the *node socket*.
*   On Linux systems, the node socket will be located at `~/.local/share/Daedalus/marlowe_pioneers/cardano-node.socket`.
*   On MacOS systems, the node socket will be located at `~/Library/Application Support/Daedalus Marlowe Pioneers/cardano-node.socket`.
*   On Windows systems, 😕 ***TO BE WRITTEN*** .

Set the shell variable `CARDANO_NODE_SOCKET_PATH` to equal the location of this socket file.

In [11]:
export CARDANO_NODE_SOCKET_PATH=~/.local/share/Daedalus/marlowe_pioneers/cardano-node.socket

Now we can check that `marlowe-cli` can communicate with the cardano node.

In [12]:
marlowe-cli util select --testnet-magic 1567 --socket-path "$CARDANO_NODE_SOCKET_PATH" $(cat my-wallet.address)

TxIn "d8c062e3d350723938cae4f1708c4c5521ad349cd345c923f52f2ccba87fe311" (TxIx 0)


Also check that `cardano-cli` can communicate with the cardano node.

In [13]:
cardano-cli query utxo --testnet-magic 1567 --address $(cat my-wallet.address)

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
d8c062e3d350723938cae4f1708c4c5521ad349cd345c923f52f2ccba87fe311     0        1000000000000 lovelace + TxOutDatumNone


## Summary

*   The Daedalus wallet and the Marlowe git repository provide the necessities for running Marlowe contracts.
*   Marlowe CLI requires access to a Cardano node such as the one bundled with the Daedalus wallet.
*   Several Cardano tools are useful for running Marlowe contracts from the command line:
    *   `marlowe-cli`
    *   `cardano-cli`
    *   `cardano-wallet`
*   One needs to create payment and signing keys in order to submit transactions to the blockchain.

## Other Lectures

*   Overview of Marlowe CLI
*   ~~Installing Marlowe CLI and Associated Tools~~
*   Running Marlowe Contracts without Blockchain Transactions
*   Running Marlowe Contacts on the Blockchain
*   Running Marlowe Contracts with the Plutus Application Backend (PAB)
*   Reference for Marlowe CLI Commands